# Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, row_number
from pyspark.sql.window import Window

spark = (SparkSession.builder.appName("cs544")
         .master("spark://main:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://main:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/17 20:08:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# ! wget https://pages.cs.wisc.edu/~harter/cs639/data/sf.zip
# ! unzip sf.zip

In [3]:
!hdfs dfs -cp sf.csv hdfs://main:9000/sf.csv

cp: `hdfs://main:9000/sf.csv': File exists


In [4]:
df = (spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load("hdfs://main:9000/sf.csv"))
columns = [col(c).alias(c.replace(" ", "")) for c in df.columns]
df.select(columns).write.format("parquet").mode("overwrite").save("hdfs://main:9000/sf.parquet")

23/03/17 20:09:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Lecture Part 1: Views and Tables

In [5]:
df = spark.read.format("parquet").load("hdfs://main:9000/sf.parquet")

In [6]:
# creating a view doesn't compute anything, it describes how to compute it as needed
# df.createTempView("calls")
df.createOrReplaceTempView("calls")

In [7]:
spark.sql("SHOW TABLES").show()   # shows both views and tables

23/03/17 20:12:06 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/03/17 20:12:06 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/03/17 20:12:13 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/03/17 20:12:13 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.24.0.2
23/03/17 20:12:14 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |    calls|       true|
+---------+---------+-----------+



## Lecture Part 2: Categories

## Lecture Part 3: Performance